# Smooth Monotonic Networks: Time comparison

Wall-clock time comparisons are always difficult, depend on implementaiton, hardware, etc.

In [ ]:
%load_ext autoreload
%autoreload 


In [ ]:
import numpy as np
import random

import torch 
import torch.nn as nn

from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score as r2
from sklearn.isotonic import IsotonicRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

import matplotlib.pyplot as plt
from tqdm.notebook import tnrange

from xgboost import XGBRegressor

from pmlayer.torch.layers import HLattice

from MonotonicNN import SmoothMonotonicNN, MonotonicNN, MonotonicNNAlt
from MonotonicNNPaperUtils import Progress, total_params

from monotonenorm import GroupSort, direct_norm, SigmaNet

import time

In [ ]:
print(torch.__version__)

In [ ]:
class FastSMM(nn.Module):
    def __init__(self, in_features, K, b_z = 1., b_t = 1., beta=-1.):
        super(FastSMM, self).__init__()
        self.in_features = in_features
        self.K = K
        self.beta_init = beta
        self.b_z = b_z
        self.b_t = b_t
        self.beta = torch.nn.Parameter(torch.ones(1), requires_grad=True)
        self.log_weight = nn.Parameter(torch.Tensor(K*K, in_features))
        self.biases = nn.Parameter(torch.Tensor(K*K))
        self.reset_parameters_()

    def reset_parameters_(self):
        nn.init.trunc_normal_(self.log_weight, std=self.b_z)
        nn.init.trunc_normal_(self.biases, std=self.b_t)
        nn.init.constant_(self.beta, self.beta_init)

    def forward(self, input):
        return forward_(input, self.log_weight.exp(), self.biases, self.beta.exp(), self.K)
    
@torch.jit.script
def forward_(input, weights, biases, beta: float, K: int):
    linear = nn.functional.linear(input, weights, biases).unfold(1, K, K) 
    linear = torch.logsumexp(linear, dim=2)
    linear = -torch.logsumexp(-linear, dim=1)/beta
    return linear

In [ ]:
def fit_torch(model, x, y, threshold=1e-3, max_iterations=1000):
    P = Progress(5, threshold=threshold)
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Rprop(model.parameters(), lr=0.01, etas=(0.5, 1.2), step_sizes=(1e-06, 50))
    dead = 0
    for epoch in range(max_iterations):
        pred_y = model(x)
        loss = loss_function(pred_y, y)
        stop = P.update(loss.item())
        loss.backward()
        optimizer.step()
        model.zero_grad()

## Multivariate experiments
Section 4.2 in the manuscript.

In [ ]:
T = 21  # number of trials, odd number for having a "median trial"
ls = 75  # lattice points (k in original paper)
ls_small = 35
K = 6  # number of SMM groups, we always use H_k = K
N_train = 100  # number of examples in training data set
N_test = 1000 # number of examples in test data set
sigma = 0.01  # noise level, feel free to vary 
width_small = K
width = K+2

In [ ]:
def generatePoly(dim=2, degree=2, sigma_train=0., sigma_test=0, N_train=100, N_test=100):
    x_train = np.random.rand(N_train, dim)
    x_test = np.random.rand(N_test, dim)
    poly = PolynomialFeatures(degree)  # includes bias
    x_poly_train = poly.fit_transform(x_train)
    x_poly_test = poly.fit_transform(x_test)
    w = np.random.rand(x_poly_train.shape[1])
    w_sum = w.sum()
    y_train = np.sum(x_poly_train * w, axis=1)/w_sum + sigma_train * np.random.normal(0, 1., N_train)
    y_test = np.sum(x_poly_test * w, axis=1)/w_sum + sigma_test * np.random.normal(0, 1., N_test)
    return x_train, y_train, x_test, y_test

In [ ]:
N_train = 500
N_test = 1000
T = 21
trees = 100
trees2 = 200

dims = [2, 4, 6]

degree = 2
ls = [10, 3, 2]
K = 6
sigma = 0.01  # noise level

In [ ]:
def do_it(method, dim, lattice_size): 
    total_t = 0
    no_params = 0
    for trial in tnrange(T):
        seed = task_id + trial*N_tasks
        random.seed(seed)
        np.random.seed(seed)
        torch.manual_seed(seed)

        lattice_sizes = list(np.ones(dim)*lattice_size)
        lattice_sizes_plus = list(np.ones(dim)*(lattice_size + 1))
        lattice_sizes_tensor = torch.tensor(lattice_sizes, dtype=torch.long)
        lattice_sizes_tensor_plus = torch.tensor(lattice_sizes_plus, dtype=torch.long)
        increasing = list(range(dim))

        x_train, y_train, x_test, y_test = generatePoly(dim, degree=degree, sigma_train=sigma, sigma_test=0., N_train=N_train, N_test=N_test)
        x_train_torch = torch.from_numpy(x_train.astype(np.float32)).clone()
        y_train_torch = torch.from_numpy(y_train.astype(np.float32)).clone()
        x_test_torch = torch.from_numpy(x_test.astype(np.float32)).clone()
        y_test_torch = torch.from_numpy(y_test.astype(np.float32)).clone()

        match method:
            case 'xgboost':             
                model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees)
                t0 = time.time()
                model.fit(x_train, y_train)
                t = time.time() - t0
                y_pred_train = model.predict(x_train)
                y_pred_test = model.predict(x_test)
            case 'xgboost_val':             
                x_train_small, x_val, y_train_small, y_val = train_test_split(x_train, y_train, test_size=.25, random_state=42)
                model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees, 
                                     early_stopping_rounds=(trees // 10), verbosity=0)
                t0 = time.time()
                model.fit(x_train_small, y_train_small, eval_set=[(x_train_small, y_train_small), (x_val, y_val)], verbose=0)
                t = time.time() - t0
                y_pred_train = model.predict(x_train)
                y_pred_test = model.predict(x_test)
            case 'xgboost2':             
                model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees2)
                t0 = time.time()
                model.fit(x_train, y_train)
                t = time.time() - t0
                y_pred_train = model.predict(x_train)
                y_pred_test = model.predict(x_test)
            case 'xgboost2_val':             
                x_train_small, x_val, y_train_small, y_val = train_test_split(x_train, y_train, test_size=.25, random_state=42)
                model = XGBRegressor(monotone_constraints=tuple(increasing), n_estimators=trees2, 
                                     early_stopping_rounds=(trees2 // 10), verbosity=0)
                t0 = time.time()
                model.fit(x_train_small, y_train_small, eval_set=[(x_train_small, y_train_small), (x_val, y_val)], verbose=0)
                y_pred_train = model.predict(x_train)
                y_pred_test = model.predict(x_test)
            case 'lattice':
                model = HLattice(dim, lattice_sizes_tensor, increasing)
                if(trial==0):
                    print(method, total_params(model), "parameters")
                    no_params = total_params(model)
                t0 = time.time()
                fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                t = time.time() - t0
                y_pred_train = model(x_train_torch).detach().numpy()
                y_pred_test = model(x_test_torch).detach().numpy()      
            case 'lattice_plus':
                model = HLattice(dim, lattice_sizes_tensor_plus, increasing)
                if(trial==0):
                    no_params = total_params(model)
                    print(method, total_params(model), "parameters")
                t0 = time.time()
                fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                t = time.time() - t0
                y_pred_train = model(x_train_torch).detach().numpy()
                y_pred_test = model(x_test_torch).detach().numpy()      
            case 'smooth':
                model = FastSMM(dim, K, beta=-1.)
                if(trial==0):
                    no_params = total_params(model)
                    print(method, total_params(model), "parameters")
                t0 = time.time()
                fit_torch(model, x_train_torch, y_train_torch)
                t = time.time() - t0
                y_pred_train = model(x_train_torch).detach().numpy()
                y_pred_test = model(x_test_torch).detach().numpy()
            case 'lip_small':
                width_small = int(dim + K)
                model = torch.nn.Sequential(
                    direct_norm(torch.nn.Linear(dim, width_small), kind="one-inf"),
                    GroupSort(width_small//2),
                    direct_norm(torch.nn.Linear(width_small, width_small), kind="inf"),
                    GroupSort(width_small//2),
                    direct_norm(torch.nn.Linear(width_small, 1), kind="inf"),
                )
                model = SigmaNet(model, sigma=1, monotone_constraints=(1,))
                if(trial==0):
                    no_params = total_params(model)
                    print(method, total_params(model), "parameters")
                t0 = time.time()
                fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                t = time.time() - t0
                y_pred_train = model(x_train_torch).detach().numpy()
                y_pred_test = model(x_test_torch).detach().numpy()
            case 'lip':
                width = int(dim + K) + 2
                model = torch.nn.Sequential(
                    direct_norm(torch.nn.Linear(dim, width), kind="one-inf"),
                    GroupSort(width//2),
                    direct_norm(torch.nn.Linear(width, width), kind="inf"),
                    GroupSort(width//2),
                    direct_norm(torch.nn.Linear(width, 1), kind="inf"),
                )
                model = SigmaNet(model, sigma=1, monotone_constraints=(1,))
                t0 = time.time()
                fit_torch(model, x_train_torch, y_train_torch.reshape(-1,1))
                t = time.time() - t0
                y_pred_train = model(x_train_torch).detach().numpy()
                y_pred_test = model(x_test_torch).detach().numpy()
                if(trial==0):
                    no_params = total_params(model)
                    print(method, total_params(model), "parameters")
        total_t += t
    return total_t, no_params


In [ ]:
methods = ['smooth', 'lattice', 'lattice_plus', 'lip_small', 'lip']
dims = [2, 4, 6]
N_methods = len(methods)
N_tasks = len(dims)


In [ ]:
times = np.zeros((N_tasks, N_methods))
params = np.zeros((N_tasks, N_methods))
for task_id, dim in enumerate(dims):
    print("dimension", dim)
    for method_id, method in enumerate(methods):
        t, p = do_it(method, dim, ls[task_id])
        print(method, "training time:", t)
        times[task_id, method_id] = t
        params[task_id, method_id] = p


In [ ]:
functions = ("$d=2$", "$d=4$", "$d=6$")
for f_id, f_name in enumerate(functions):
    print(f_name, end= " & ")
    for i, (t, p) in enumerate(zip(times[f_id, :], params[f_id, ])):
        print("{:.2f}".format(t), " & ", int(p), end='')
        if(i < len(methods)-1):
            print(" & ", end='')
        else:
            print(" \\\\")